# Feature Generators <a name="head"></a>

In this tutorial, we will look at generating features from a database of organic donor-acceptor molecules from the [Computational Materials Repository](https://cmrdb.fysik.dtu.dk/?project=solar). This has been downloaded in the [ase-db](https://wiki.fysik.dtu.dk/ase/ase/db/db.html#module-ase.db) format so first off we load the atoms objects and get a target property. Then we convert the atoms objects into a feature array and test out a couple of different models.

This tutorial will give an indication of one way in which it is possible to handle atoms objects of different sizes. In particular, we focus on a feature set that scales with the number of atoms. We pad the feature vectors to a constant size to overcome this problem.

## Table of Contents
[(Back to top)](#head)

-   [Requirements](#requirements)
-   [Data Setup](#data-setup)
-   [Feature Generation](#feature-generation)
-   [Predictions](#predictions)
-   [Cross-validation](#cross-validation)

## Requirements <a name="requirements"></a>
[(Back to top)](#head)

-   [AtoML](https://gitlab.com/atoml/AtoML)
-   [ASE](https://wiki.fysik.dtu.dk/ase/)
-   [numpy](http://www.numpy.org/)
-   [matplotlib](https://matplotlib.org/)
-   [seaborn](http://seaborn.pydata.org/index.html)

## Data Setup <a name="data-setup"></a>
[(Back to top)](#head)

First, we need to import some functions.

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ase.db

from atoml.fingerprint import StandardFingerprintGenerator
from atoml.fingerprint.setup import return_fpv, get_atomic_types
from atoml.regression import RidgeRegression, GaussianProcess
from atoml.cross_validation import Hierarchy
from atoml.utilities.cost_function import get_error

plt.rcParams['figure.figsize'] = (20.0, 10.0)

We then need to extract the atoms objects from the db, as this is the format that AtoML will require to generate the feature vectors. At this point, the target values are also compiled. For this tutorial, we will use the ground state energies as targets.

In [ ]:
# Connect the ase-db.
db = ase.db.connect('../../data/solar.db')

# Compile a list of atoms and target values.
alist = []
targets = []
for row in db.select():
    alist.append(row.toatoms())
    targets.append(row.Energy)

# Analyze the size of molecules in the db.
print('pulled {} molecules from db'.format(len(alist)))
size = []
for a in alist:
    size.append(len(a))

print('min: {0}, mean: {1:.0f}, max: {2} molecule size'.format(
    min(size), sum(size)/len(size), max(size)))

Here we can see that there are a total of ~5400 molecules in the db ranging from 26 to 294 atoms.

## Feature Generation <a name="feature-generation"></a>
[(Back to top)](#head)

It can be necessary to work out the full range of elements that need to be accounted for in the model. The feature generator tries to work out the range of elements to account for based on the maximum composition. However, explicitly specifying this is more robust.

In [ ]:
atypes = get_atomic_types(train_candidates=alist, test_candidates=None)
print('Atom numbers present in data: {}'.format(atypes))

We then generate the feature array for all the atoms objects. The `return_fpv()` function takes the list of atoms objects and the type of features to generate.

In [ ]:
fpv = StandardFingerprintGenerator(atom_types=atypes)
features = return_fpv(alist, [fpv.eigenspectrum_fpv,
                              fpv.composition_fpv])

print('{} shape feature matrix'.format(np.shape(features)))

After this, we can analyze the distribution of the feature sets. In the following, we see a large number of features in the latter half of the vectors tend to be zero.

In [ ]:
dif = np.max(features, axis=0) - np.min(features, axis=0)
np.place(dif, dif == 0., [1.])
scaled = features.copy() / dif
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=1000, center="dark")
sns.heatmap(scaled, cmap=cmap)
plt.show()

Next, we shuffle the data and split the features and targets into a single testing and training dataset for our initial studies. To start with we can take approximately a fifth of the data to train on (1000 data points), and test on the remaining data.

In [ ]:
# Shuffle the order of the data.
targets = np.reshape(targets, (len(targets), 1))
data = np.concatenate((features, targets), axis=1)
np.random.shuffle(data)
features = data[:, :-1]
targets = np.reshape(data[:, -1:], (np.shape(data)[0],))

# Divide up the data into a test and training set.
train_size = 1000
train_features = features[:train_size, :]
test_features = features[train_size:, :]
train_targets = targets[:train_size]
test_targets = targets[train_size:]

print('training feature size: {0}, training target size: {1}'.format(
    np.shape(train_features), np.shape(train_targets)))
print('testing feature size: {0}, testing target size: {1}'.format(
    np.shape(test_features), np.shape(test_targets)))

## Predictions <a name="predictions"></a>
[(Back to top)](#head)

We can now try predictions with ridge regression to start. This clearly performs very well with this data. Based on these results, it is unlikely that you would consider moving to more complex models.

In [ ]:
# Set up the ridge regression function.
rr = RidgeRegression(W2=None, Vh=None, cv='loocv')
b = rr.find_optimal_regularization(X=train_features, Y=train_targets)
coef = rr.RR(X=train_features, Y=train_targets, omega2=b)[0]

# Test the model.
sumd = 0.
err = []
pred = []
for tf, tt in zip(test_features, test_targets):
    p = np.dot(coef, tf)
    pred.append(p)
    sumd += (p - tt) ** 2
    e = ((p - tt) ** 2) ** 0.5
    err.append(e)
error = (sumd / len(test_features)) ** 0.5

print(error)

plt.plot(test_targets, pred, 'o', alpha=0.5)
plt.show()

However, for the purposes of this tutorial, we then train a Gaussian processes regression model to test. In this case, we set up a kernel dictionary that has both the squared exponential and linear kernels. The initial parameters defined in the kernel aren't so important at this stage as they are all optimized when the model is trained.

In [ ]:
kdict = {
    'k1': {
        'type': 'gaussian', 'width': 1., 'scaling': 1., 'dimension': 'single'},
    'k2' : {
        'type': 'linear', 'scaling': 1.},
    }
gp = GaussianProcess(train_fp=train_features, train_target=train_targets,
                     kernel_dict=kdict, regularization=1e-2,
                     optimize_hyperparameters=True, scale_data=True)

pred = gp.predict(test_fp=test_features)

error = get_error(pred['prediction'],
                  test_targets)['rmse_average']

print(error)

plt.plot(test_targets, pred['prediction'], 'o', alpha=0.5)
plt.show()

Here we see that the Gaussian process performs slightly worse than the simple ridge regression model. This is to be expected when we are trying to model linear data with a non-linear model. However, the inclusion of the linear kernel results in a good prediction error. If the squared exponential kernel were to be removed from the above example, the resulting model would be the same as the ridge regression model, just trained with the Gaussian process.

## Cross-validation <a name="cross-validation"></a>
[(Back to top)](#head)

We can use the hierarchy cross-validation module to investigate how the model performs with different data sizes. In the following, we set up a prediction function. As the ridge regression function performs well, we just redefine this. The prediction function should take in the training and testing data and return a dictionary in the form `{'result': list, 'size': list}`.

In [ ]:
def rr_predict(train_features, train_targets, test_features, test_targets):
    """Function to perform the RR predictions."""
    data = {}

    # Set up the ridge regression function.
    rr = RidgeRegression(W2=None, Vh=None, cv='loocv')
    b = rr.find_optimal_regularization(X=train_features, Y=train_targets)
    coef = rr.RR(X=train_features, Y=train_targets, omega2=b)[0]

    # Test the model.
    sumd = 0.
    err = []
    for tf, tt in zip(test_features, test_targets):
        p = np.dot(coef, tf)
        sumd += (p - tt) ** 2
        e = ((p - tt) ** 2) ** 0.5
        err.append(e)
    error = (sumd / len(test_features)) ** 0.5

    data['result'] = error
    data['size'] = len(train_targets)

    return data

We then run the cv and display the resulting learning curve.

In [ ]:
# Initialize hierarchy cv class.
hv = Hierarchy(db_name='test.sqlite', file_name='hierarchy')
# Convert features and targets to simple db format.
hv.todb(features=features, targets=targets)
# Split the data into subsets.
ind = hv.split_index(min_split=100, max_split=2000)

# Make the predictions for each subset.
pred = hv.split_predict(index_split=ind, predict=rr_predict)

# Get mean error at each data size.
means, meane = hv.transform_output(pred)

# Plot the results.
plt.plot(pred[1], pred[0], 'o', c='b', alpha=0.5)
plt.plot(means, meane, '-', alpha=0.9, c='black')
plt.show()

Finally, the output is removed.

In [ ]:
os.remove('hierarchy.pickle')
os.remove('test.sqlite')